# Les variables binaires en optimisation 
Tentative de clarification simplifiée, et étude par estelle derrien

<b>CREATION EN COURS </b>

- Elles permettent d'établir des contraintes logiques (Ou, Si/Alors, Et...), d'autres contraintes logique existent, telles que les contraintes de multiplicité, par exemple.
- Valeurs : Elles prennent la valeur 0 ou 1 (non ou oui).
- Intérêt : Sélectionner une variable de décision , pénaliser une variable de décision, sélectionner une contrainte, discriminer une contrainte, établir une condition.
- Utilisation : Directement dans la fonction objectif, ou avec les contraintes.
- Déclaration : Elle diffère selon les solveurs, elle est plus facile dans certains.
- Semblable à : IF/THEN ( Si/Alors en Anglais ), Semblable à un principe ON/OFF pour les machines. Equivaut à un 'trigger' en Anglais, un déclencheur.
- Relaté à : Les méthodes <b>"Big M"</b>.
- Permettent souvent de : Eviter de faire une optimisation non linéaire dans le cadre des "Piecewize constraints".
- Permettent d'attribuer une <b>pénalité</b> à une variable de décision, si sa valeur dépasse un certain seuil.
- On évite d'utiliser la fonctionnalité de limites (Bounds) sur les variables de décision souvent fournies avec le solveur, <b>en même temps que</b> les variables binaires, c'est soit l'un,soit l'autre.

Liens :

Pour comprendre les contraintes logiques et la méthode BigM dans une certaine mesure :

https://ocw.mit.edu/courses/15-053-optimization-methods-in-management-science-spring-2013/resources/mit15_053s13_lec11/

https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf

http://www.yzuda.org/Useful_Links/optimization/if-then-else-02.html

https://benalexkeen.com/linear-programming-with-python-and-pulp-part-6/

https://stackoverflow.com/questions/58825442/how-can-i-write-an-if-condition-for-my-decision-variable-for-mixed-integer-linea

#  Les Exemples simples que l'on va tenter de traiter avec les solveurs :

Exemple 1/ Si la production de x dépasse 20, alors la contrainte z de coût de maintenance de 10 euros s'applique, sinon, elle ne s'applique pas.

Exemple 2/ Si la production de l'objet x dépasse 100, alors la contrainte z de coût maintenance de 10 euros s'applique, et si la production de l'objet x dépasse 200, alors, la contrainte k de coût de maintenance de 20 euros s'applique, mais pas la contrainte z de 10 euros.
Si la production de l'objet y dépasse 50, alors, la contrainte t de coût de maintenance de 15 euros s'applique sinon, elle ne s'applique pas.

Exemple 3/ Si un objet X1(variable de décision) dépasse 20kgs, alors le container Z n'est pas utilisé, c'est le container Y qui est utilisé.

Exemple 4/ Si un élément chimique X1 (variable de décision) dépasse un seuil de 20grammes, alors, une pénalité lui est appliquée, ce qui privilégie l'élement chimique X2 dans l'optimisation finale.

Exemple 5/ Déclencher une machine X2 si la production sur la machine X1 est > 1000, sinon, ne pas la déclencher (Mode ON/OFF)



## Exemple 1
<b>Si la production de x dépasse 20, alors la contrainte z de coût maintenance de 10 euros s'applique, sinon, elle ne s'applique pas.</b>

On s'entraine sur un programme linéaire simple avec Pulp .
La création de la variable binaire z se passe en deux temps
- 1. On instancie la variable binaire
- 2. On crée une contrainte qui mets en jeu cette variable binaire

Le problème : L'usine produit 2  objets x et y et les vends resp 10.5 euros et 8.5 euros, quand la production de l'objet x dépasse 20 unités, alors un coût de maintenance de 10 euros est soustrait à notre profit.  Comment modéliser ce problème simple avec les solveurs ?

In [38]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
  
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# -----------------------------------
# On définit nos constantes
#
# -----------------------------------

# On spécifie le cout de maintenance
cout_maintenance = 10

# On spécifie le seuil de déclenchement de la maintenance
declencheur_maintenance = 20

# -----------------------------------
# On définit nos variables de décision
#
# -----------------------------------
  
# On Crée les variables de décision du problème , x et y sont des objets que l'usine produit
x = p.LpVariable("x", lowBound = 0, cat='Integer')   # Create a variable x >= 0 
y = p.LpVariable("y", lowBound = 0, cat='Integer')   # Create a variable y >= 0 

# Comme on a besoin d'appliquer un cout de maintenance conditionnel, on a besoin 
# de définir une variable binaire qui va se déclencher si la production de x est supérieure
# à 20 objets !
z = p.LpVariable("z", lowBound=0, cat='Binary')
z.setInitialValue(0) # On tente d'initialiser la variable à 0

# -----------------------------------
# On définit la fonction objectif
#
# -----------------------------------
  
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
# Ici, x est vendu 10.5 euros et y 8.5 euros, le coût de 10 euros de maintenance
#  est soustrait seulement si z est positive.
Lp_prob +=  10.5 * x + 8.5 * y - cout_maintenance * z


# -----------------------------------
# On définit nos contraintes
#
# ----------------------------------- 

# Heures de travail au mois
# Ca prends 3 heures de crée un objet x, et 2 heures de créer un objet z
Lp_prob += 3 * x + 2 * y  <= 420

# Il faut produire au minimum ce nombre d'éléments  :
Lp_prob += x  >= 100
Lp_prob += y  >= 40

# La contrainte binaire qu'on doit faire:

# On utilise la méthode " BigM "
# On vérifie que cela fonctionne en changeant le signe, on voit que la variable binaire passe bien 
# de 1 à 0 et que le coût est appliqué dans un cas, et pas dans l'autre dans la fonction objectif.

M = 1e6 # M se calcule selon une certaine méthode , voir plus bas.

# si x > 20 alors z = 1 s'écrit comme cela avec PULP
Lp_prob += z >= (x - 20)/M


# -----------------------------------
# On résouds avec le solveur
#
# -----------------------------------

# Afficher le problème linéaire
# print(Lp_prob) 
status = Lp_prob.solve()   # Exécuter le solver
# print(p.LpStatus[status])   # Le statut de la solution

# Afficher la solution :
print(p.value(x),"Objets produits x")
print(p.value(y) , "Objets produits y"  )
print(p.value(z) , "La valeur de la variable binaire utilisée ou pas , 0 ou 1"  )
print(p.value(Lp_prob.objective) ,"est notre profit" )


100.0 Objets produits x
60.0 Objets produits y
1.0 La valeur de la variable binaire utilisée ou pas , 0 ou 1
1550.0 est notre profit


Maintenant, on va essayer avec le solveur DocPlex, qu ipermet d'écrire les contraintes binaires plus simplement

In [39]:

import cplex
import docplex.mp
from docplex.mp.model import Model

# On crée notre modèle
model = Model(name='LP_example', log_output=True)

# On crée nos variables de décision
x = model.integer_var(name='x')
y = model.integer_var(name='y')
z = model.binary_var(name='z')

# On crée la fonction objectif
model.maximize(10.5 * x + 8.5 * y - z * cout_maintenance)

# On crée les contraintes
model.add_constraint(x  >= 100)
model.add_constraint(y >= 40)
model.add_constraint(3 * x + 2 * y  <= 420)

# On spécifie le déclencheur de la contrainte z, 
# qui dit que si la production de x dépasse 20, 
# alors le coût de 10 euros est soustrait dans la fonction objectif
#if then constraint
model.add_constraint(model.if_then(x >= declencheur_maintenance, z == 1))


model.print_information() 
sol_model = model.solve()
model.print_solution()


Model: LP_example
 - number of variables: 4
   - binary=2, integer=2, continuous=0
 - number of constraints: 5
   - linear=3, indicator=1, equiv=1
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 2 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Found incumbent of value 1380.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynam

Donc, on voit qu'avec Pulp, et surement d'autre solveurs, on doit utiliser une astuce appelée BigM , parce que ils n'ont pas de fonctionnalité IF/THEN native.
 Voici la traduction de l'auteur qui a fourni un bon exemple de BigM sur StackOverflow (Kabdulla et Alfer):

 Le terme de recherche que vous recherchez est "variable indicatrice" ou "contrainte big-M".

Autant que je sache, PULP ne prend pas directement en charge les variables indicatrices, donc une contrainte big-M est la voie à suivre.

A Simple Example: x1 <= 0 IF x2 > 2


In [40]:
from pulp import *

prob = LpProblem("MILP", LpMaximize)
x1 = LpVariable("x1", lowBound=0, upBound=10, cat = 'Continuous')
x2 = LpVariable("x2", lowBound=0, upBound=10, cat = 'Continuous')

prob += 0.5*x1 + x2, "Objective Function"

b1 = LpVariable("b1", cat='Binary')

M1 = 1e6
prob += b1 >= (x1 - 2)/M1

M2 = 1e3
prob += x2 <= M2*(1 - b1)

status = prob.solve()
print(LpStatus[status])
print(x1.varValue, x2.varValue, b1.varValue, pulp.value(prob.objective))

Optimal
2.0 10.0 0.0 11.0


Nous voulons qu'une contrainte x1 <= 0 existe quand x2 > 2. Quand x2 <= 2 aucune telle contrainte n'existe (x1 peut être positif ou négatif).

Nous créons d'abord une variable binaire :

In [41]:
b1 = LpVariable("b1", cat='Binary')

Choisissez ceci pour représenter la condition x2 > 2. Le moyen le plus simple d'y parvenir en ajoutant une contrainte :



In [42]:
M1 = 1e6
prob += b1 >= (x2 - 2)/M1

Ici, M1 est la valeur du grand M. Il doit être choisi de telle sorte que pour la plus grande valeur possible de x2, l'expression (x2-2)/M soit <=1. Il doit être aussi petit que possible pour éviter les problèmes numériques/de mise à l'échelle. Ici, une valeur de 10 fonctionnerait (x2 a une limite supérieure de 10).

Pour comprendre comment fonctionne cette contrainte, pensez aux cas, pour x2<=2, le côté droit est au plus 0, et n'a donc aucun effet (borne inférieure d'une variable binaire déjà définie sur 0). Cependant, si x2> 2, le côté droit forcera b1 à être supérieur à 0 - et en tant que variable binaire, il sera forcé à être 1.

Enfin, nous devons construire la contrainte requise :


In [43]:
M2 = 1e3
prob += x1 <= M2*(b1 - 1)

Encore une fois pour comprendre le fonctionnement de cette contrainte, considérons les cas, si b1 est vrai (1) la contrainte est active et devient : x1 <= 0. Si b1 est faux ('0') la contrainte devient x1 <= M2, à condition que M2 est suffisamment grand, cela n'aura aucun effet (ici, il pourrait être aussi petit que 10 car x1 a déjà une limite supérieure de 10.

Dans le code complet ci-dessus, si vous faites varier le coefficient de x1 dans la fonction objectif, vous devriez remarquer que b1 est activé/désactivé et la contrainte supplémentaire appliquée à x1 comme prévu.

## Exemple 2
<b>
Si la production de l'objet x dépasse 100, alors la contrainte z de coût maintenance de 10 euros s'applique, et si la production de l'objet x dépasse 200, alors, la contrainte k de coût de maintenance de 20 euros s'applique, mais pas la contrainte z de 10 euros.

Si la production de l'objet y dépasse 50, alors, la contrainte t de coût de maintenance de 15 euros s'applique sinon, elle ne s'applique pas.

</b>

In [44]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
  
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# -----------------------------------
# On définit nos constantes
#
# -----------------------------------

# On spécifie les cout de maintenance
cout_maintenance_100 = 10
cout_maintenance_200 = 20
cout_maintenance_50 = 15

# On spécifie le seuil de déclenchement de la maintenance
declencheur_maintenance = 20

# -----------------------------------
# On définit nos variables de décision
#
# -----------------------------------
  
# On Crée les variables de décision du problème , x et y sont des objets que l'usine produit
x = p.LpVariable("x", lowBound = 0, cat='Integer')   # Create a variable x >= 0 
y = p.LpVariable("y", lowBound = 0, cat='Integer')   # Create a variable y >= 0 

z = p.LpVariable("z", lowBound=0, cat='Binary')
t = p.LpVariable("t", lowBound=0, cat='Binary')


# -----------------------------------
# On définit la fonction objectif
#
# -----------------------------------
  
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
# Ici, x est vendu 10.5 euros et y 8.5 euros, les couts sont soustraits 
# en fonction de la valeur des variables binaires ( 0 ou 1).
# Par exemple, si la variable binaire k est égale à 0, alors il n'y a pas de couts 
# de maintenance 200 puisque la multiplication cout_maintenance_200 * k fait 0

Lp_prob +=  10.5 * x + 8.5 * y - cout_maintenance_100 * z - cout_maintenance_200 * k - cout_maintenance_50 *  t


# -----------------------------------
# On définit nos contraintes
#
# ----------------------------------- 

# Heures de travail au mois
# Ca prends 3 heures de crée un objet x, et 2 heures de créer un objet z
Lp_prob += 3 * x + 2 * y  <= 7000

# Il faut produire au minimum ce nombre d'éléments  :
Lp_prob += x  >= 1000
Lp_prob += y  >= 400

# La contrainte binaire qu'on doit faire:

# On utilise la méthode " BigM "
# On vérifie que cela fonctionne en changeant le signe, on voit que la variable binaire passe bien 
# de 1 à 0 et que le coût est appliqué dans un cas, et pas dans l'autre dans la fonction objectif.

M1 = 1e6 
M2 = 1e6 
M3 = 1e6 

Lp_prob += z >= (x - 100)/M1
Lp_prob += z >= (x - 200)/M1
Lp_prob += z >= (x - 200)/M1


# -----------------------------------
# On résouds avec le solveur
#
# -----------------------------------

# Afficher le problème linéaire
# print(Lp_prob) 
status = Lp_prob.solve()   # Exécuter le solver
# print(p.LpStatus[status])   # Le statut de la solution

# Afficher la solution :
print(p.value(x),"Objets produits x")
print(p.value(y) , "Objets produits y"  )
print(p.value(z) , "La valeur de la variable binaire utilisée ou pas , 0 ou 1"  )
print(p.value(Lp_prob.objective) ,"est notre profit" )

NameError: name 'k' is not defined